# Chapter 18 - Gaussian Processes

Any model that is linear in its parameters with a Gaussian distribution over the parameters is a **Gaussian process**. This class spans discrete models, including random walks, and autoregressive processes, as well as continuous models, including Bayesian linear regression models, polynomials, Fourier series, radial basis functions, and even neural networks with an infinite number of hidden units.

## 18.1. Introduction to Gaussian Processes

Unlike the weights of neural networks, which are relatively uninterpretable, the gaussian proceses provide a mechanism for directly reasoning about the high-level properties of functions that could fit the data.

![](../imgs/ch18/gp-observed-data.svg)

Suppose we have a dataset shown in the figure above of regression targets (outputs) $y$ at inputs $x$. To fit the data with a Gaussian process, we start by specifying a *prior distribution* over what types of functions we may believe to be reasonable, some of which are shown in the figure below.

![](../imgs/ch18/gp-sample-prior-functions.svg)

We are not looking for functions that fit the dataset exactly, but rather for specifying reasonable high-level properties of the solutions, such as how quickly they vary with inputs.

Once we condition on data, we can use this prior to infer a *posterior distribution* over functions that could fit the data. The figure below shows some possible posterior functions.

![](../imgs/ch18/gp-sample-posterior-functions.svg)

Each of these posterior functions are entirely consistent with the data, perfectly running through each observation. In order to use these posterior samples to make predictions, we can average the values of every possible sample function from the posterior, to create the blue sruve in the figure below.

![](../imgs/ch18/gp-posterior-samples.svg)

We do not have to take an infinite number of samples to compute this epxectation.

We also want to know how confident we should be in our predictions. Intuitively, we should have more uncertainty where there is more variability in the sample posterior functions, as this tells us there are many more possible values the true function could take. This type of uncertainty is called *epistemic uncertainty*, which is the *reducible uncertainty* associated with lack of information. As we acquire more data, this type of uncertainty disappears, as there will be increasingly fewer solutions consistent with what we observe.

Similar to the posterior mean, we can compute the *posterior variance* in closed form. The shaded region in the figure below shows the two-time posterior standard deviation on either side of the mean, creating a *credible interval* that has a 95% probability of containing the true value of the function for any input $x$.

![](../imgs/ch18/gp-posterior-samples-95.svg)

The properties of the Gaussian process that we fit the data are strongly controlled by the *covariance function*, also known as the *kernel*. The covariance function we used is called the *RBF (Radial Basis Function) kernel*, which has the form
\begin{split}
k_{\textrm{RBF}}(x,x') = \textrm{Cov}(f(x),f(x')) = a^2 \exp\left(-\frac{1}{2\ell^2}||x-x'||^2\right)
\end{split}

The *hyperparameters* of this kernel are interpretable:
* the *amplitude* $a$ controls the vertical scale over which the function is varying. Larger $a$ means larger function values.
* the *length-scale* $\ell$ controls the rate of variation of the function. Larger $\ell$ means the function varies more slowly.

At $||x-x'|| = \ell$, the covariance between a pair of function values is $a^2\exp(-0.5)$. At larger distances than $\ell$, the values of the function values becomes nearly uncorrelated, which means that if we want to make a prediction at a point $x_*$, then function values with input $x$ such that $||x-x_*|| > \ell$ will not have a strong effect on our predictions.

The generalization performance of a Gaussian process will depend on having reasonable values for these hyperparameters. Fortunately, there is a robust and automatic way to specify these hyperparameters, using *maximum likelihood*.

The essense of a Gaussian process states that any collection of function values $f(x_1),\dots,f(x_n)$, indexed by any collection of inputs $x_1,\dots,x_n$, has a joint multivariate Gaussian distribution. The mean vector $\mu$ of this distribution is given by a *mean function*, which is typically taken to be a constant or zero. The covariance matrix of this distribution is given by the *kernel* evaluated at all pairs of the inputs $x$.

\begin{split}
\begin{bmatrix}
f(x) \\f(x_1) \\ \vdots \\ f(x_n) \end{bmatrix}\sim \mathcal{N}\left(\mu, \begin{bmatrix}k(x,x) & k(x, x_1) & \dots & k(x,x_n) \\ k(x_1,x) & k(x_1,x_1) & \dots & k(x_1,x_n) \\ \vdots & \vdots & \ddots & \vdots \\ k(x_n, x) & k(x_n, x_1) & \dots & k(x_n,x_n) 
\end{bmatrix}\right)
\end{split}

This specifies a GP prior. We can compute the conditional distribution of $f(x)$ for any $x$ given $f(x_1),\dots,f(x_n)$, the function values we have observed. This conditional distribution is called the *posterior*, and it is what we use to make predictions.

In particular,
\begin{split}
f(x) | f(x_1), \dots, f(x_n) \sim \mathcal{N}(m,s^2)
\end{split}
where
\begin{aligned}
m &= k(x,x_{1:n}) k(x_{1:n},x_{1:n})^{-1} f(x_{1:n}) \\
s^2 &= k(x,x) - k(x,x_{1:n})k(x_{1:n},x_{1:n})^{-1}k(x,x_{1:n})
\end{aligned}
where $k(x,x_{1:n})$ is a $1\times n$ vector formed by evaluating $k(x, x_i)$ for $i=1,\dots,n$, and $k(x_{1:n},x_{1:n})$ is an $n\times n$ matrix formed by evaluating $k(x_i,x_j)$ for $i,j=1,\dots,n$.

The value $m$ is waht we can use as a point predictor for any $x$, and $s^2$ is what we use for uncertainty: if we want to create an interval with a 95% probability that $f(x)$ is in the interval, we would use $m\pm 2s$.

Suppose we observe a single datapoint, $f(x_1)$ and we want to determine the value of $f(x)$ at some $x$. Because $f(x)$ is described by a Gaussian process, we know the joint distribution over $f(x)$ and $f(x_1)$ is a Gaussian:

\begin{split}
\begin{bmatrix}
f(x) \\
f(x_1) \\
\end{bmatrix}
\sim
\mathcal{N}\left(\mu,
\begin{bmatrix}
k(x,x) & k(x, x_1) \\
k(x_1,x) & k(x_1,x_1)
\end{bmatrix}
\right)\end{split}

The off-diagonal expression $k(x,x_1) = k(x_1, x)$ tells how correlated the function values will be - how strongly determined $f(x)$ will be form $f(x_1)$. If we use a large length-scale $\ell$, relative to $||x-x_1||$, then the function values will be highly correlated.

## 18.2. Gaussian Process Priors

In [1]:
import numpy as np
from scipy.spatial import distance_matrix

from d2l import torch as d2l

d2l.set_figsize()

### 18.2.1. Definition